In [209]:
from __future__ import print_function
import numpy as np
import random as rnd
from sys import exit
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [214]:
class Salesman:
    def __init__(self, c: int, a: int, alpha:float=.1, beta:float=1.1, gamma:float=.1, generate:bool=True, path:str="cities_example.txt"):
        self.cities = c
        self.start = rnd.randint(0, c - 1)
        self.ants = a
        self.k = 10 * c
        
        if generate:
            self.d = self.init_cities(2 * 5)
            np.savetxt(path, self.d, fmt="%i")
        else:
            try:
                self.d = np.loadtxt(path, dtype=int)
            except OSError:
                exit("Error: If you specify the path argument with generate=False, then you have to create a [path].txt file following cities_example.txt format. It should be a square matrix for distances between cities with zeroes on the diagonal.")
        self.p = np.zeros((c, c))
        self.c = np.zeros((self.k, c))
        # wtf
        self.q = 1
        self.gamma = gamma
        
        # wtf
        self.curiosity = 0.1
        self.alpha = alpha
        self.beta = beta
        
        print("Initialized TSP with map:")
        print(self.d)
        print(f"Starting from town #{self.start}")
    
    def init_cities(self, maxd:int) -> np.ndarray:
        d = np.random.randint(-maxd, maxd, (self.cities, self.cities))
        d = np.abs((d + d.T) // 2) + 1
        np.fill_diagonal(d, 0)
        return d
        
        
    def run(self, it:int, verbose:bool=False) -> None:
        minroad = ([], 1000000)
        for i in range(it):
            roads = [Ant(self.d, self.p, self.start, self.alpha, self.beta).walk() for _ in range(self.ants)]
            self.update_pheromones(roads, verbose)
            
            roadslength = [sum([c[1] for c in r]) for r in roads]
            roads = [[path[0] for path in r] for r in roads]
            bestroad = (np.argmin(roadslength), min(roadslength))
            
            minroad = (roads[bestroad[0]], bestroad[1]) if bestroad[1] < minroad[1] else minroad
            if verbose:
                print(*roads[bestroad[0]], bestroad[1])
        print(f"Best road : {minroad}")
    
    def update_pheromones(self, roads:list, verbose:bool=False) -> None:
        # evaporation
        self.p *= .9
        for road in roads:
            d = sum([c[1] for c in road])
            s = self.start
            
            for c in road[1:]:
                self.p[s][c[0]] += self.q / d
                self.p[c[0]][s] += self.q / d
                s = c[0]
                
            if verbose:
                print(d, self.p)
    
class Ant:
    def __init__(self, c: np.ndarray, p: np.ndarray, s:int, a:float, b:float):
        self.cities = c
        self.p_map = p
        self.to_visit = [v for v in range(c.shape[0]) if not v == s]
        self.alpha = a
        self.beta = b
        
        self.road = [(s, 0)]
    
    def choose_road(self) -> int:
        crt = self.road[-1][0]
        ps = {}
        for c, _ in enumerate(self.to_visit):
            intensity = self.p_map[crt][c]
            desirability = 1 / self.cities[crt][c]
            ps[c] = intensity * self.alpha * desirability * self.beta
        pt = sum(ps.values())
        ps = {p: ps[p]/pt for p in ps}
        try:
            return np.random.choice(a=list(ps.keys()), p=list(ps.values()))
        except ValueError:
            return rnd.randint(0, len(self.to_visit) - 1)
    
    def walk(self) -> list:
        while len(self.to_visit) > 0:
            crt = self.road[-1][0]
            i = self.choose_road()
            c = self.to_visit.pop(i)
            self.road.append((c, self.cities[crt][c]))
        return self.road

In [229]:
# ants
# nb cities
# start (depending on nb cities)
# alpha
# beta
# iterations
# gamma
# curiosity
# ants = interact(lambda x: x, x=widgets.IntSlider(min=2, max=30, step=1, value=10, description="Ants count"))
# cities = interact(lambda x: x, x=widgets.IntSlider(min=1, max=30, step=1, value=10, description="Cities count"))
# start = interact(lambda x: x, x=widgets.IntSlider(min=0, max=30, step=1, value=10, description="Start from"))
# alpha = interact(lambda x: x, x=widgets.FloatSlider(min=0, max=1, step=.01, value=10, description="𝛼"))
# beta = interact(lambda x: x, x=widgets.FloatSlider(min=1, max=10, step=.01, value=10, description="𝛽"))
# iterations = interact(lambda x: x, x=widgets.IntSlider(min=1, max=1000, step=1, value=10, description="Iterations"))
# gamma = interact(lambda x: x, x=widgets.FloatSlider(min=-10, max=30, step=1, value=10, description="𝛾"))
# curiosity = interact(lambda x: x, x=widgets.FloatSlider(min=-10, max=30, step=1, value=10, description="Curiosity"))

salesman = Salesman(5, 20, generate=False)
salesman.run(100, verbose=False)

interactive(children=(IntSlider(value=10, description='Ants count', max=30, min=2), Output()), _dom_classes=('…

interactive(children=(IntSlider(value=10, description='Cities count', max=30, min=1), Output()), _dom_classes=…

interactive(children=(IntSlider(value=10, description='Start from', max=30), Output()), _dom_classes=('widget-…

interactive(children=(FloatSlider(value=1.0, description='𝛼', max=1.0, step=0.01), Output()), _dom_classes=('w…

interactive(children=(FloatSlider(value=10.0, description='𝛽', max=10.0, min=1.0, step=0.01), Output()), _dom_…

interactive(children=(IntSlider(value=10, description='Iterations', max=1000, min=1), Output()), _dom_classes=…

interactive(children=(FloatSlider(value=10.0, description='𝛾', max=30.0, min=-10.0, step=1.0), Output()), _dom…

interactive(children=(FloatSlider(value=10.0, description='Curiosity', max=30.0, min=-10.0, step=1.0), Output(…

Initialized TSP with map:
[[ 0  4  2  2 11]
 [ 4  0  2  1  3]
 [ 2  2  0  8  6]
 [ 2  1  8  0  3]
 [11  3  6  3  0]]
Starting from town #2
Best road : ([2, 0, 3, 1, 4], 8)


/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:81: RuntimeWarning: divide by zero encountered in long_scalars
/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:82: RuntimeWarning: invalid value encountered in double_scalars
/anaconda3/envs/ml/lib/python3.7/site-packages/ipykernel_launcher.py:84: RuntimeWarning: invalid value encountered in double_scalars
